In [ ]:
import os
import re
from tqdm import tqdm
import tensorflow as tf
import tensorflow_io as tfio
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D
from tensorflow.keras.metrics import TruePositives, TrueNegatives, FalsePositives, FalseNegatives
from sklearn.model_selection import train_test_split
import librosa
from pydub import AudioSegment
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
path_spoof = "idrd/Training_Data/spoof/"
path_human = "idrd/Training_Data/human/"
path_test = "idrd/Testing_Data/"

spoof_list_path = [path_spoof + i for i in os.listdir(path_spoof)]
human_list_path = [path_human + i for i in os.listdir(path_human)]
test_list_path = [path_test + i for i in os.listdir(path_test)]

In [ ]:
def write_slice_audio(path_load, path_write, seconds):
    audio = AudioSegment.from_wav(path_load)
    n = re.search(r'\d+', path_load).group(0)
    start = 0
    end = seconds*1000 # ms
    for i in range(len(audio)//(seconds*1000)):
        slice_audio = audio[start:end]
        start = end
        end += seconds*1000
        slice_audio.export(f'{path_write}{n}.{i+1}.wav', format="wav")

In [ ]:
[write_slice_audio(i, 'idrd/Slice_Training_Data/spoof/', 2) for i in tqdm(spoof_list_path)]
[write_slice_audio(i, 'idrd/Slice_Training_Data/human/', 2) for i in tqdm(human_list_path)]
[write_slice_audio(i, 'idrd/Slice_Testing_Data/', 2) for i in tqdm(test_list_path)]

In [ ]:
path_spoof = "idrd/Slice_Training_Data/spoof/"
path_human = "idrd/Slice_Training_Data/human/"
path_test = "idrd/Slice_Testing_Data/"

spoof_list_path = [path_spoof + i for i in os.listdir(path_spoof)]
human_list_path = [path_human + i for i in os.listdir(path_human)]
test_list_path = [path_test + i for i in os.listdir(path_test)]

spoof_lables = [0]*len(spoof_list_path)
human_lables = [1]*len(human_list_path)

spoof_df = pd.DataFrame(list(zip(spoof_list_path, spoof_lables)))
human_df = pd.DataFrame(list(zip(human_list_path, human_lables)))
data_df = pd.concat([spoof_df,human_df], ignore_index=True)

data_df = shuffle(data_df)
train_data, test_data = train_test_split(data_df, train_size=0.7, stratify=data_df[1], random_state=0)
test_data, val_data = train_test_split(test_data, train_size=0.5, stratify=test_data[1], random_state=0)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE


def get_dataset(df):
    file_path_ds = tf.data.Dataset.from_tensor_slices(df[0])
    label_ds = tf.data.Dataset.from_tensor_slices(df[1])
    return tf.data.Dataset.zip((file_path_ds, label_ds))


def mel_spec(sounds):
    y = tfio.audio.AudioIOTensor(sounds).to_tensor()
    y = tf.squeeze(y, -1)
    y = tf.cast(y, tf.float32)
    y = y/32768.0
    spectrogram = tfio.audio.spectrogram(y, nfft=512, window=512, stride=256)
    mel_spectrogram = tfio.audio.melscale(spectrogram, rate=16000, mels=128, fmin=0, fmax=8000)
    dbscale_mel_spectrogram = tfio.audio.dbscale(mel_spectrogram, top_db=80)
    return tf.expand_dims(dbscale_mel_spectrogram, -1)/255


def prepare_for_training(ds, shuffle_buffer_size=1024, batch_size=256):
    # Randomly shuffle (file_path, label) dataset
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    # Load and decode audio from file paths
    ds = ds.map(lambda x,y: (tf.py_function(func=load_and_mel_spec, inp=[x], Tout=tf.float32), y,), num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch_size)
    # Prefetch
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

In [ ]:
ds = get_dataset(train_data)
train_ds = prepare_for_training(ds)

ds = get_dataset(val_data)
val_ds = prepare_for_training(ds)

batch_size = 64

In [ ]:
for i in train_ds:
    print(i[0].shape)
    break

In [ ]:
input_shape=(125, 128, 1)
input = keras.Input(shape=input_shape)
f = Conv2D(64, kernel_size=(3,3), activation='elu')(input)
f = Conv2D(32, kernel_size=(3,3), activation='elu')(f)
f = Conv2D(16, kernel_size=(5,5), activation='elu')(f)
f = Flatten()(f)
d = Dense(320, activation='elu')(f)
d = Dense(160, activation='elu')(d)
d = Dense(1, activation='sigmoid')(d)

model = keras.Model(inputs=input, outputs=d)
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=["accuracy", "AUC", TruePositives(), TrueNegatives(), FalsePositives(), FalseNegatives()])


In [ ]:
history = model.fit(train_ds, batch_size=256, epochs=10, validation_data=val_ds, shuffle=True)